In [1]:
using MambaModels, Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;
line[:x] = convert(Array{Float64,1}, df2[:weight_c]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

352×2 Array{Float64,2}:
 1.0    2.83512
 1.0   -8.50468
 1.0  -13.1256 
 1.0    8.05143
 1.0   -3.71361
 1.0   18.0021 
 1.0   -6.74701
 1.0   10.4895 
 1.0  -10.1206 
 1.0    9.49725
 ⋮             
 1.0    2.89182
 1.0   -5.58468
 1.0   -3.94041
 1.0   -4.16721
 1.0    2.04133
 1.0  -10.7443 
 1.0    7.17259
 1.0    9.07201
 1.0    7.54114

Model Specification

In [3]:
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

Object of type "Model"
-------------------------------------------------------------------------------
beta:
A monitored node of type "0-element ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
y:
An unmonitored node of type "0-element ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "ScalarStochastic"
NaN


Initial Values

In [4]:
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

3-element Array{Dict{Symbol,Any},1}:
 Dict(:beta=>[188.575, -7.45719],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>44.3773)
 Dict(:beta=>[-34.9119, 4.48206],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>49.3563)
 Dict(:beta=>[303.128, 4.9742],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>22.0901)  

Tuning Parameters

In [5]:
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

0.1

Define sampling scheme

In [6]:
scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

Object of type "Model"
-------------------------------------------------------------------------------
beta:
A monitored node of type "0-element ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
y:
An unmonitored node of type "0-element ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "ScalarStochastic"
NaN


MCMC Simulation

In [7]:
chn = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:00:19 of 0:00:19 remaining]
Chain 1:  10% [0:00:03 of 0:00:03 remaining]
Chain 1:  20% [0:00:03 of 0:00:03 remaining]
Chain 1:  30% [0:00:02 of 0:00:03 remaining]
Chain 1:  40% [0:00:02 of 0:00:03 remaining]
Chain 1:  50% [0:00:02 of 0:00:03 remaining]
Chain 1:  60% [0:00:01 of 0:00:03 remaining]
Chain 1:  70% [0:00:01 of 0:00:03 remaining]
Chain 1:  80% [0:00:01 of 0:00:03 remaining]
Chain 1:  90% [0:00:00 of 0:00:03 remaining]
Chain 1: 100% [0:00:00 of 0:00:03 remaining]

Chain 2:   0% [0:00:04 of 0:00:04 remaining]
Chain 2:  10% [0:00:03 of 0:00:03 remaining]
Chain 2:  20% [0:00:03 of 0:00:03 remaining]
Chain 2:  30% [0:00:02 of 0:00:03 remaining]
Chain 2:  40% [0:00:02 of 0:00:03 remaining]
Chain 2:  50% [0:00:02 of 0:00:03 remaining]
Chain 2:  60% [0:00:01 of 0:00:03 remaining]
Chain 2:  70% [0:00:01 of 0:00:03 remaining]
Chain 2:  80% [0:00:01 of 0:00:03 remaining]
Chain 2:  90% [0:00:00 of 0:00:03 remaining]
Cha

Object of type "ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[28.4293 154.413 0.968027; 23.9516 154.47 0.953985; … ; 25.2441 154.733 0.860705; 25.1193 154.733 0.860705]

[24.1879 154.658 0.918832; 24.0408 154.346 0.91763; … ; 29.4184 154.974 0.850955; 29.2723 154.976 0.872956]

[28.2272 154.759 0.918082; 27.9374 154.257 0.937243; … ; 26.667 154.732 0.886742; 26.3659 154.582 0.882689]

Show draws summary

In [8]:
describe(chn)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
           Mean        SD        Naive SE        MCSE        ESS  
     s2  26.138793 2.01920729 0.01228850424 0.01950587222 9000.000
beta[1] 154.596386 0.26666244 0.00162285590 0.00339166517 6181.555
beta[2]   0.904958 0.04219299 0.00025677836 0.00031833564 9000.000

Quantiles:
            2.5%         25.0%       50.0%        75.0%       97.5%   
     s2  22.47957981  24.73817568  26.0156453  27.42342638  30.4184854
beta[1] 154.08558277 154.41128183 154.5905624 154.77770219 155.1374824
beta[2]   0.82217774   0.87652145   0.9050816   0.93315194   0.9871476



End of `m4.1m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*